In [1]:
import yaml
import json
import graphviz

graph = yaml.safe_load(open("graph.yml", "r"))

# Process Node

> 其实想一下， 一个node上真实运行的内容还是一个个pod和db（数据库其实以容器形式也可以叫pod）

In [3]:
from textwrap import indent
import os

useful_type = ["DB", "Docker"]
nodes = [v for v in graph if v["class"] == "node" and v["type"] in useful_type]


dot = graphviz.Digraph("graph", format="svg")
dot_deploy = graphviz.Digraph("deployment", format="svg")

for v in nodes:
    params = v["params"]
    for p in params:
        p_v = params[p]

        for value in p_v:
            true_id = v["id"].replace(f"{{{p}}}", value)
            dot.node(true_id)

print(dot.source)

useful_type =  ["call"]
edges = [v for v in graph if v["class"] == "edge" and v["type"] in useful_type]

deployments = [v for v in graph if v["class"] == "edge" and v["type"] == "deployment"]

# docker -> os
deploy_map = {}
for _dep in deployments:
    if "docker" not in _dep["params"] or "os" not in _dep["params"]: continue
    for src, dst in zip(_dep["params"]["docker"], _dep["params"]["os"]):
        if "docker" in src and "os" in dst:
            deploy_map[src] = dst
            dot_deploy.edge(src, dst)
        
json.dump(deploy_map, open(os.path.join("output", "graph.json"), "w"), indent=4)

for e in edges:
    srcs = e["params"]["src"]
    dsts = e["params"]["dst"]
    for src, dst in zip(srcs, dsts):
        if "os" not in src and "os" not in dst:
            dot.edge(src, dst)
        elif "os" in src and "docker" in dst:
            dot.edge(src, deploy_map[dst])


dot.render(filename="graph", directory="output")
dot_deploy.render(filename="deployment", directory="output")

digraph "graph" {
	docker_001
	docker_002
	docker_003
	docker_004
	docker_005
	docker_006
	docker_007
	docker_008
	db_003
	db_007
	db_009
}



'output/deployment.svg'

'output/graph.svg'